# Factory Pattern in LLM Applications: Multi-Provider AI Client System

## Overview

This notebook demonstrates how to use the **Factory Pattern** to create a flexible multi-provider AI client system. We'll build a system that can dynamically create different AI clients (OpenAI, Anthropic, Google, Local models) based on requirements.

### Why Factory Pattern for Multi-LLM Systems?

- **Dynamic Creation**: Choose AI providers at runtime based on cost, performance, or availability
- **Decoupled Design**: Client code doesn't need to know specific implementation details
- **Easy Extension**: Add new providers without modifying existing code
- **Configuration-Driven**: Switch providers through configuration changes

Let's start by exploring how traditional object creation becomes problematic with multiple AI providers, then see how the Factory Pattern elegantly solves these issues.

## 🚀 Setting Up the Foundation

First, let's define our base interfaces that all AI providers will implement:

In [ ]:
from abc import ABC, abstractmethod
from enum import Enum
from typing import Dict, List, Optional, Union
import json
import time
import random

# Define provider types
class ProviderType(Enum):
    OPENAI = "openai"
    ANTHROPIC = "anthropic"
    GOOGLE = "google"
    LOCAL = "local"
    MOCK = "mock"  # For demonstration

# Base AI Client Interface
class AIClient(ABC):
    """Abstract base class for all AI clients"""
    
    @abstractmethod
    def generate_text(self, prompt: str, **kwargs) -> Dict:
        """Generate text response from prompt"""
        pass
    
    @abstractmethod
    def get_model_info(self) -> Dict:
        """Get information about the model"""
        pass
    
    @abstractmethod
    def estimate_cost(self, text: str) -> float:
        """Estimate cost for processing given text"""
        pass

print("✅ Base interfaces defined!")

## 🛠️ The Problem: Without Factory Pattern

Let's see what happens when we create AI clients directly without using the Factory pattern:

In [ ]:
# ❌ PROBLEMATIC APPROACH: Direct instantiation everywhere

class ProblematicAIService:
    def __init__(self, provider_name: str):
        # This approach has multiple problems:
        # 1. Tight coupling
        # 2. Hard to extend
        # 3. Configuration scattered
        # 4. Difficult to test
        
        if provider_name == "openai":
            self.client = OpenAIClient(api_key="sk-...", model="gpt-4")
        elif provider_name == "anthropic":
            self.client = AnthropicClient(api_key="ant_...", model="claude-3-opus")
        elif provider_name == "google":
            self.client = GoogleClient(api_key="AI...", model="gemini-pro")
        else:
            raise ValueError(f"Unknown provider: {provider_name}")
    
    def chat(self, message: str):
        return self.client.generate_text(message)

# Problems with this approach:
print("❌ Problems with direct instantiation:")
print("1. Every place that creates clients needs to know all provider details")
print("2. Adding a new provider requires modifying this class")
print("3. Configuration is hardcoded and scattered")
print("4. Hard to unit test with mocks")
print("5. Violates Open/Closed Principle")

## ✨ The Solution: Factory Pattern Implementation

Now let's implement concrete AI clients that follow our interface:

In [ ]:
# Concrete AI Client Implementations

class OpenAIClient(AIClient):
    def __init__(self, api_key: str, model: str = "gpt-4", **kwargs):
        self.api_key = api_key
        self.model = model
        self.cost_per_1k_tokens = 0.03
        
    def generate_text(self, prompt: str, **kwargs) -> Dict:
        # Simulate OpenAI API call
        response_time = random.uniform(1, 3)
        time.sleep(0.1)  # Simulate network delay
        
        return {
            "text": f"[OpenAI {self.model}] Response to: {prompt[:50]}...",
            "provider": "openai",
            "model": self.model,
            "tokens_used": len(prompt.split()) + 20,
            "response_time": response_time,
            "cost": self.estimate_cost(prompt)
        }
    
    def get_model_info(self) -> Dict:
        return {
            "provider": "OpenAI",
            "model": self.model,
            "max_tokens": 4096,
            "cost_per_1k": self.cost_per_1k_tokens,
            "strengths": ["General purpose", "Code generation", "Reasoning"]
        }
    
    def estimate_cost(self, text: str) -> float:
        tokens = len(text.split()) * 1.3  # Rough token estimation
        return (tokens / 1000) * self.cost_per_1k_tokens


class AnthropicClient(AIClient):
    def __init__(self, api_key: str, model: str = "claude-3-opus-20240229", **kwargs):
        self.api_key = api_key
        self.model = model
        self.cost_per_1k_tokens = 0.015
        
    def generate_text(self, prompt: str, **kwargs) -> Dict:
        response_time = random.uniform(0.8, 2.5)
        time.sleep(0.1)
        
        return {
            "text": f"[Anthropic {self.model}] Thoughtful response to: {prompt[:50]}...",
            "provider": "anthropic",
            "model": self.model,
            "tokens_used": len(prompt.split()) + 25,
            "response_time": response_time,
            "cost": self.estimate_cost(prompt)
        }
    
    def get_model_info(self) -> Dict:
        return {
            "provider": "Anthropic",
            "model": self.model,
            "max_tokens": 200000,
            "cost_per_1k": self.cost_per_1k_tokens,
            "strengths": ["Safety", "Long context", "Analysis"]
        }
    
    def estimate_cost(self, text: str) -> float:
        tokens = len(text.split()) * 1.3
        return (tokens / 1000) * self.cost_per_1k_tokens


class GoogleClient(AIClient):
    def __init__(self, api_key: str, model: str = "gemini-1.5-pro", **kwargs):
        self.api_key = api_key
        self.model = model
        self.cost_per_1k_tokens = 0.0025
        
    def generate_text(self, prompt: str, **kwargs) -> Dict:
        response_time = random.uniform(0.5, 2.0)
        time.sleep(0.1)
        
        return {
            "text": f"[Google {self.model}] Efficient response to: {prompt[:50]}...",
            "provider": "google",
            "model": self.model,
            "tokens_used": len(prompt.split()) + 15,
            "response_time": response_time,
            "cost": self.estimate_cost(prompt)
        }
    
    def get_model_info(self) -> Dict:
        return {
            "provider": "Google",
            "model": self.model,
            "max_tokens": 1000000,
            "cost_per_1k": self.cost_per_1k_tokens,
            "strengths": ["Fast", "Cost-effective", "Multimodal"]
        }
    
    def estimate_cost(self, text: str) -> float:
        tokens = len(text.split()) * 1.3
        return (tokens / 1000) * self.cost_per_1k_tokens


class MockClient(AIClient):
    """Mock client for testing and development"""
    def __init__(self, **kwargs):
        self.model = "mock-model"
        self.cost_per_1k_tokens = 0.0
        
    def generate_text(self, prompt: str, **kwargs) -> Dict:
        return {
            "text": f"[MOCK] Instant response to: {prompt[:50]}...",
            "provider": "mock",
            "model": self.model,
            "tokens_used": len(prompt.split()) + 10,
            "response_time": 0.01,
            "cost": 0.0
        }
    
    def get_model_info(self) -> Dict:
        return {
            "provider": "Mock",
            "model": self.model,
            "max_tokens": 9999999,
            "cost_per_1k": 0.0,
            "strengths": ["Testing", "Development", "Free"]
        }
    
    def estimate_cost(self, text: str) -> float:
        return 0.0

print("✅ All concrete AI clients implemented!")

## 🏭 The Factory: Smart AI Client Creation

Now for the star of the show - our Factory class that intelligently creates AI clients:

In [ ]:
class AIClientFactory:
    """Factory for creating AI clients based on various criteria"""
    
    # Registry of available clients
    _clients = {
        ProviderType.OPENAI: OpenAIClient,
        ProviderType.ANTHROPIC: AnthropicClient,
        ProviderType.GOOGLE: GoogleClient,
        ProviderType.MOCK: MockClient
    }
    
    # Default configurations for each provider
    _default_configs = {
        ProviderType.OPENAI: {
            "api_key": "sk-demo-key",
            "model": "gpt-4",
            "temperature": 0.7
        },
        ProviderType.ANTHROPIC: {
            "api_key": "ant-demo-key", 
            "model": "claude-3-opus-20240229",
            "temperature": 0.7
        },
        ProviderType.GOOGLE: {
            "api_key": "AI-demo-key",
            "model": "gemini-1.5-pro",
            "temperature": 0.7
        },
        ProviderType.MOCK: {}
    }
    
    @classmethod
    def create_client(cls, provider_type: ProviderType, **kwargs) -> AIClient:
        """Create a client for the specified provider type"""
        if provider_type not in cls._clients:
            raise ValueError(f"Unknown provider type: {provider_type}")
        
        # Merge default config with provided kwargs
        config = cls._default_configs[provider_type].copy()
        config.update(kwargs)
        
        # Create and return the client
        client_class = cls._clients[provider_type]
        return client_class(**config)
    
    @classmethod
    def create_client_by_name(cls, provider_name: str, **kwargs) -> AIClient:
        """Create a client by provider name string"""
        try:
            provider_type = ProviderType(provider_name.lower())
            return cls.create_client(provider_type, **kwargs)
        except ValueError:
            raise ValueError(f"Unknown provider name: {provider_name}")
    
    @classmethod
    def create_best_client_for_task(cls, task_requirements: Dict) -> AIClient:
        """Intelligently choose the best client based on task requirements"""
        
        # Cost-sensitive tasks
        if task_requirements.get("cost_priority", False):
            print("🎯 Selecting cost-optimized provider: Google")
            return cls.create_client(ProviderType.GOOGLE)
        
        # Speed-critical tasks
        if task_requirements.get("speed_priority", False):
            print("🎯 Selecting speed-optimized provider: Google")
            return cls.create_client(ProviderType.GOOGLE)
        
        # Safety-critical tasks
        if task_requirements.get("safety_priority", False):
            print("🎯 Selecting safety-focused provider: Anthropic")
            return cls.create_client(ProviderType.ANTHROPIC)
        
        # Long context tasks
        if task_requirements.get("long_context", False):
            print("🎯 Selecting long-context provider: Anthropic")
            return cls.create_client(ProviderType.ANTHROPIC)
        
        # Development/testing
        if task_requirements.get("development", False):
            print("🎯 Selecting development provider: Mock")
            return cls.create_client(ProviderType.MOCK)
        
        # Default: balanced choice
        print("🎯 Selecting balanced provider: OpenAI")
        return cls.create_client(ProviderType.OPENAI)
    
    @classmethod
    def register_client(cls, provider_type: ProviderType, client_class, default_config: Dict):
        """Register a new client type (for extensibility)"""
        cls._clients[provider_type] = client_class
        cls._default_configs[provider_type] = default_config
        print(f"✅ Registered new client: {provider_type}")
    
    @classmethod
    def get_available_providers(cls) -> List[str]:
        """Get list of available providers"""
        return [provider.value for provider in cls._clients.keys()]
    
    @classmethod
    def compare_providers(cls, prompt: str) -> Dict:
        """Compare all providers for a given prompt"""
        results = {}
        
        for provider_type in cls._clients.keys():
            client = cls.create_client(provider_type)
            info = client.get_model_info()
            cost = client.estimate_cost(prompt)
            
            results[provider_type.value] = {
                "model_info": info,
                "estimated_cost": cost
            }
        
        return results

print("🏭 AI Client Factory created!")
print(f"📋 Available providers: {AIClientFactory.get_available_providers()}")

## 🎮 Demo 1: Basic Factory Usage

Let's see the Factory Pattern in action with basic client creation:

In [ ]:
print("=== DEMO 1: Basic Factory Usage ===")

# Creating clients using different methods
print("\n1. Creating clients by ProviderType enum:")
openai_client = AIClientFactory.create_client(ProviderType.OPENAI)
print(f"   ✅ Created: {openai_client.get_model_info()['provider']} client")

print("\n2. Creating clients by name string:")
anthropic_client = AIClientFactory.create_client_by_name("anthropic")
print(f"   ✅ Created: {anthropic_client.get_model_info()['provider']} client")

print("\n3. Creating clients with custom configuration:")
custom_client = AIClientFactory.create_client(
    ProviderType.GOOGLE, 
    model="gemini-1.5-flash",  # Override default model
    temperature=0.2
)
print(f"   ✅ Created: {custom_client.get_model_info()['provider']} client with custom config")

# Test all clients with the same prompt
test_prompt = "Explain quantum computing in simple terms"
clients = [openai_client, anthropic_client, custom_client]

print(f"\n4. Testing all clients with prompt: '{test_prompt}'")
for i, client in enumerate(clients, 1):
    response = client.generate_text(test_prompt)
    print(f"   Client {i} ({response['provider']}): {response['text'][:60]}...")
    print(f"             Cost: ${response['cost']:.4f}, Time: {response['response_time']:.2f}s")

## 🧠 Demo 2: Intelligent Provider Selection

The real power of our Factory: automatically choosing the best provider based on task requirements!

In [ ]:
print("=== DEMO 2: Intelligent Provider Selection ===")

# Different task scenarios
scenarios = [
    {
        "name": "Budget-Conscious Startup",
        "requirements": {"cost_priority": True},
        "prompt": "Write a product description for our app"
    },
    {
        "name": "Real-Time Chat Application", 
        "requirements": {"speed_priority": True},
        "prompt": "Generate a quick response to user greeting"
    },
    {
        "name": "Healthcare AI Assistant",
        "requirements": {"safety_priority": True},
        "prompt": "Provide general wellness information"
    },
    {
        "name": "Legal Document Analyzer",
        "requirements": {"long_context": True},
        "prompt": "Analyze this complex legal document"
    },
    {
        "name": "Development Environment",
        "requirements": {"development": True},
        "prompt": "Test prompt for development"
    }
]

for scenario in scenarios:
    print(f"\n📋 Scenario: {scenario['name']}")
    print(f"   Requirements: {scenario['requirements']}")
    
    # Factory automatically selects best provider
    client = AIClientFactory.create_best_client_for_task(scenario['requirements'])
    
    # Generate response
    response = client.generate_text(scenario['prompt'])
    model_info = client.get_model_info()
    
    print(f"   Selected: {model_info['provider']} ({model_info['model']})")
    print(f"   Strengths: {', '.join(model_info['strengths'])}")
    print(f"   Response: {response['text'][:70]}...")
    print(f"   Cost: ${response['cost']:.4f}, Time: {response['response_time']:.2f}s")

## 📊 Demo 3: Provider Comparison

Let's compare all providers side-by-side for the same task:

In [ ]:
print("=== DEMO 3: Provider Comparison ===")

comparison_prompt = "Write a Python function to calculate fibonacci numbers"
print(f"\n🔍 Comparing all providers for: '{comparison_prompt}'")

# Get comparison data
comparison = AIClientFactory.compare_providers(comparison_prompt)

print("\n" + "="*80)
print(f"{'Provider':<12} {'Model':<25} {'Cost/1K':<10} {'Est.Cost':<10} {'Strengths':<25}")
print("="*80)

for provider_name, data in comparison.items():
    info = data['model_info']
    cost = data['estimated_cost']
    
    strengths = ', '.join(info['strengths'][:2])  # Show first 2 strengths
    
    print(f"{info['provider']:<12} {info['model']:<25} ${info['cost_per_1k']:<9.4f} ${cost:<9.4f} {strengths:<25}")

print("="*80)

# Now let's actually test each provider
print("\n🚀 Live Response Comparison:")
responses = []

for provider_name in comparison.keys():
    if provider_name != "mock":  # Skip mock for more realistic comparison
        client = AIClientFactory.create_client_by_name(provider_name)
        response = client.generate_text(comparison_prompt)
        responses.append(response)
        
        print(f"\n🤖 {response['provider'].upper()}:")
        print(f"   Response: {response['text']}")
        print(f"   Metrics: {response['tokens_used']} tokens, ${response['cost']:.4f}, {response['response_time']:.2f}s")

# Summary statistics
if responses:
    avg_cost = sum(r['cost'] for r in responses) / len(responses)
    avg_time = sum(r['response_time'] for r in responses) / len(responses)
    fastest = min(responses, key=lambda r: r['response_time'])
    cheapest = min(responses, key=lambda r: r['cost'])
    
    print(f"\n📈 Summary Statistics:")
    print(f"   Average cost: ${avg_cost:.4f}")
    print(f"   Average time: {avg_time:.2f}s")
    print(f"   Fastest: {fastest['provider']} ({fastest['response_time']:.2f}s)")
    print(f"   Cheapest: {cheapest['provider']} (${cheapest['cost']:.4f})")

## 🔧 Demo 4: Extensibility - Adding New Providers

One of the Factory Pattern's greatest strengths is extensibility. Let's add a new provider without modifying existing code:

In [ ]:
print("=== DEMO 4: Adding New Providers ===")

# Let's create a new provider - a local model
class LocalLlamaClient(AIClient):
    """Example local model client (like Ollama)"""
    
    def __init__(self, model: str = "llama2-7b", **kwargs):
        self.model = model
        self.cost_per_1k_tokens = 0.0  # Local models are "free" after setup
        
    def generate_text(self, prompt: str, **kwargs) -> Dict:
        # Local models are slower but free
        response_time = random.uniform(3, 8)  # Slower than cloud APIs
        time.sleep(0.2)  # Simulate processing time
        
        return {
            "text": f"[Local {self.model}] Private response to: {prompt[:50]}...",
            "provider": "local_llama",
            "model": self.model,
            "tokens_used": len(prompt.split()) + 30,
            "response_time": response_time,
            "cost": 0.0
        }
    
    def get_model_info(self) -> Dict:
        return {
            "provider": "Local Llama",
            "model": self.model,
            "max_tokens": 4096,
            "cost_per_1k": 0.0,
            "strengths": ["Private", "Free", "Customizable"]
        }
    
    def estimate_cost(self, text: str) -> float:
        return 0.0

# Register the new provider with our factory
class ExtendedProviderType(Enum):
    LOCAL_LLAMA = "local_llama"

# Register the new client type
AIClientFactory.register_client(
    ExtendedProviderType.LOCAL_LLAMA,
    LocalLlamaClient,
    {"model": "llama2-13b"}
)

print("\n🆕 New provider registered! Testing it out...")

# Create and test the new client
local_client = AIClientFactory.create_client(ExtendedProviderType.LOCAL_LLAMA)
response = local_client.generate_text("What are the benefits of running models locally?")

print(f"\n🤖 New Provider Test:")
print(f"   Provider: {response['provider']}")
print(f"   Response: {response['text']}")
print(f"   Cost: ${response['cost']:.4f} (Free!)")
print(f"   Time: {response['response_time']:.2f}s (Slower but private)")

# Show updated provider list
print(f"\n📋 Updated available providers: {AIClientFactory.get_available_providers()}")

# The beauty of the Factory Pattern: no existing code needed to change!
print("\n✨ Key insight: We added a new provider without modifying ANY existing code!")
print("   This demonstrates the Open/Closed Principle - open for extension, closed for modification.")

## 🛡️ Demo 5: Production-Ready Features

Let's implement some production-ready features like failover, retry logic, and environment-based configuration:

In [ ]:
print("=== DEMO 5: Production-Ready Factory ===")

class ProductionAIClientFactory(AIClientFactory):
    """Enhanced factory with production features"""
    
    @classmethod
    def create_resilient_client(cls, primary_provider: ProviderType, 
                              fallback_providers: List[ProviderType] = None,
                              max_retries: int = 3) -> 'ResilientAIClient':
        """Create a client with automatic failover and retry logic"""
        
        if fallback_providers is None:
            fallback_providers = [ProviderType.MOCK]  # Always have mock as final fallback
        
        return ResilientAIClient(primary_provider, fallback_providers, max_retries, cls)
    
    @classmethod
    def create_from_environment(cls, env: str = "development") -> AIClient:
        """Create client based on environment configuration"""
        
        env_configs = {
            "development": {
                "provider": ProviderType.MOCK,
                "reason": "Fast, free, no API keys needed"
            },
            "testing": {
                "provider": ProviderType.MOCK,
                "reason": "Consistent, deterministic responses"
            },
            "staging": {
                "provider": ProviderType.GOOGLE,
                "reason": "Cost-effective for pre-production testing"
            },
            "production": {
                "provider": ProviderType.OPENAI,
                "reason": "High quality, reliable service"
            }
        }
        
        config = env_configs.get(env, env_configs["development"])
        print(f"🌍 Environment: {env} -> {config['provider'].value} ({config['reason']})")
        
        return cls.create_client(config["provider"])


class ResilientAIClient:
    """A resilient client that handles failures gracefully"""
    
    def __init__(self, primary_provider: ProviderType, 
                 fallback_providers: List[ProviderType],
                 max_retries: int,
                 factory_class):
        self.primary_provider = primary_provider
        self.fallback_providers = fallback_providers
        self.max_retries = max_retries
        self.factory = factory_class
        
        # Create all clients upfront
        self.clients = {}
        all_providers = [primary_provider] + fallback_providers
        for provider in all_providers:
            self.clients[provider] = self.factory.create_client(provider)
    
    def generate_text(self, prompt: str, **kwargs) -> Dict:
        """Generate text with automatic failover"""
        
        providers_to_try = [self.primary_provider] + self.fallback_providers
        
        for provider in providers_to_try:
            client = self.clients[provider]
            
            for attempt in range(self.max_retries):
                try:
                    print(f"🔄 Attempting {provider.value} (attempt {attempt + 1}/{self.max_retries})")
                    
                    # Simulate occasional failures for demo
                    if provider != ProviderType.MOCK and random.random() < 0.3:  # 30% failure rate
                        raise Exception(f"Simulated {provider.value} API failure")
                    
                    response = client.generate_text(prompt, **kwargs)
                    response["resilient_info"] = {
                        "provider_used": provider.value,
                        "attempt_number": attempt + 1,
                        "is_fallback": provider != self.primary_provider
                    }
                    
                    print(f"✅ Success with {provider.value}!")
                    return response
                    
                except Exception as e:
                    print(f"❌ {provider.value} failed (attempt {attempt + 1}): {str(e)}")
                    if attempt == self.max_retries - 1:  # Last attempt with this provider
                        print(f"🔄 Moving to next provider...")
                        break
                    time.sleep(0.1)  # Brief pause before retry
        
        raise Exception("All providers and retries exhausted")
    
    def get_model_info(self) -> Dict:
        return self.clients[self.primary_provider].get_model_info()

print("\n🏗️ Production Factory Features:")

# Demo 1: Environment-based configuration
print("\n1. Environment-based client creation:")
for env in ["development", "testing", "staging", "production"]:
    client = ProductionAIClientFactory.create_from_environment(env)
    info = client.get_model_info()
    print(f"   {env:>11}: {info['provider']} - {info['model']}")

# Demo 2: Resilient client with failover
print("\n2. Resilient client with failover:")
resilient_client = ProductionAIClientFactory.create_resilient_client(
    primary_provider=ProviderType.OPENAI,
    fallback_providers=[ProviderType.GOOGLE, ProviderType.ANTHROPIC, ProviderType.MOCK],
    max_retries=2
)

# Test resilient client
print("\n🔄 Testing resilient client (with simulated failures):")
try:
    response = resilient_client.generate_text("What is machine learning?")
    resilient_info = response["resilient_info"]
    
    print(f"\n✅ Final result:")
    print(f"   Provider used: {resilient_info['provider_used']}")
    print(f"   Attempt number: {resilient_info['attempt_number']}")
    print(f"   Was fallback: {resilient_info['is_fallback']}")
    print(f"   Response: {response['text'][:60]}...")
    
except Exception as e:
    print(f"❌ All providers failed: {e}")

print("\n🎯 Production Benefits:")
print("   ✅ Environment-specific configurations")
print("   ✅ Automatic failover and retry logic")
print("   ✅ Graceful degradation")
print("   ✅ High availability")

## 🎯 Real-World Application: Multi-Provider Chat Service

Let's build a complete chat service that uses our Factory Pattern to provide intelligent provider selection:

In [ ]:
print("=== REAL-WORLD APPLICATION: Multi-Provider Chat Service ===")

class IntelligentChatService:
    """A chat service that intelligently selects AI providers based on various factors"""
    
    def __init__(self):
        self.factory = ProductionAIClientFactory()
        self.conversation_history = []
        self.user_preferences = {}
        
    def set_user_preferences(self, user_id: str, preferences: Dict):
        """Set user-specific preferences"""
        self.user_preferences[user_id] = preferences
        print(f"✅ Preferences set for user {user_id}: {preferences}")
    
    def analyze_message(self, message: str) -> Dict:
        """Analyze message to determine optimal provider"""
        analysis = {
            "word_count": len(message.split()),
            "estimated_tokens": len(message.split()) * 1.3,
            "is_coding_related": any(word in message.lower() for word in 
                                   ["code", "function", "python", "javascript", "programming", "debug"]),
            "is_creative": any(word in message.lower() for word in 
                             ["story", "poem", "creative", "imagine", "write"]),
            "is_analytical": any(word in message.lower() for word in 
                               ["analyze", "compare", "evaluate", "research", "study"]),
            "complexity": "high" if len(message.split()) > 50 else "medium" if len(message.split()) > 20 else "low"
        }
        
        return analysis
    
    def select_optimal_provider(self, message: str, user_id: str = None) -> AIClient:
        """Select the best provider based on message analysis and user preferences"""
        
        analysis = self.analyze_message(message)
        user_prefs = self.user_preferences.get(user_id, {})
        
        print(f"\n🔍 Message Analysis:")
        print(f"   Word count: {analysis['word_count']}")
        print(f"   Complexity: {analysis['complexity']}")
        print(f"   Coding related: {analysis['is_coding_related']}")
        print(f"   Creative: {analysis['is_creative']}")
        print(f"   Analytical: {analysis['is_analytical']}")
        
        requirements = {}
        
        # User preferences override
        if user_prefs.get("cost_conscious"):
            requirements["cost_priority"] = True
            print("   💰 User prefers cost optimization")
        
        if user_prefs.get("speed_important"):
            requirements["speed_priority"] = True
            print("   ⚡ User prefers speed")
        
        if user_prefs.get("privacy_focused"):
            # Would select local model in real implementation
            print("   🔒 User prefers privacy (would select local model)")
        
        # Content-based selection
        if analysis["is_coding_related"]:
            print("   💻 Detected coding task - selecting code-optimized provider")
            # OpenAI is often considered good for code
        
        if analysis["complexity"] == "high":
            requirements["quality_priority"] = True
            print("   🧠 High complexity - selecting advanced provider")
        
        if analysis["is_analytical"]:
            requirements["safety_priority"] = True
            print("   📊 Analytical task - selecting safety-focused provider")
        
        return self.factory.create_best_client_for_task(requirements)
    
    def chat(self, message: str, user_id: str = "default") -> Dict:
        """Main chat method"""
        
        print(f"\n💬 User {user_id}: {message}")
        
        # Select optimal provider
        client = self.select_optimal_provider(message, user_id)
        
        # Generate response
        response = client.generate_text(message)
        model_info = client.get_model_info()
        
        # Store in conversation history
        self.conversation_history.append({
            "user_id": user_id,
            "message": message,
            "response": response["text"],
            "provider": response["provider"],
            "cost": response["cost"],
            "timestamp": time.time()
        })
        
        result = {
            "response": response["text"],
            "provider_used": response["provider"],
            "model": model_info["model"],
            "cost": response["cost"],
            "response_time": response["response_time"],
            "provider_strengths": model_info["strengths"]
        }
        
        print(f"🤖 {model_info['provider']}: {result['response']}")
        print(f"   📊 Stats: ${result['cost']:.4f}, {result['response_time']:.2f}s, {result['provider_used']}")
        
        return result
    
    def get_conversation_stats(self) -> Dict:
        """Get statistics about the conversation"""
        if not self.conversation_history:
            return {"message": "No conversations yet"}
        
        total_cost = sum(conv["cost"] for conv in self.conversation_history)
        provider_usage = {}
        
        for conv in self.conversation_history:
            provider = conv["provider"]
            provider_usage[provider] = provider_usage.get(provider, 0) + 1
        
        return {
            "total_messages": len(self.conversation_history),
            "total_cost": total_cost,
            "provider_usage": provider_usage,
            "average_cost_per_message": total_cost / len(self.conversation_history)
        }

# Create and test the chat service
chat_service = IntelligentChatService()

# Set up different user profiles
chat_service.set_user_preferences("budget_user", {"cost_conscious": True})
chat_service.set_user_preferences("speed_user", {"speed_important": True})
chat_service.set_user_preferences("privacy_user", {"privacy_focused": True})

# Test different scenarios
test_conversations = [
    ("budget_user", "What's the weather like today?"),
    ("speed_user", "Quick question: what's 2+2?"),
    ("privacy_user", "Help me write a personal email to my friend"),
    ("default", "Write a Python function to sort a list of dictionaries by a specific key"),
    ("default", "Analyze the pros and cons of renewable energy sources in detail, considering economic, environmental, and technological factors"),
    ("default", "Write a creative short story about a robot learning to paint")
]

for user_id, message in test_conversations:
    chat_service.chat(message, user_id)
    print("-" * 60)

# Show conversation statistics
stats = chat_service.get_conversation_stats()
print(f"\n📊 CONVERSATION STATISTICS:")
print(f"   Total messages: {stats['total_messages']}")
print(f"   Total cost: ${stats['total_cost']:.4f}")
print(f"   Average cost per message: ${stats['average_cost_per_message']:.4f}")
print(f"   Provider usage: {stats['provider_usage']}")

## 🎓 Key Takeaways: Factory Pattern Benefits

Let's summarize what we've learned about using the Factory Pattern in LLM applications:

In [ ]:
print("=== KEY TAKEAWAYS: Factory Pattern Benefits ===")

benefits = {
    "🎯 Dynamic Selection": [
        "Choose AI providers based on runtime requirements",
        "Optimize for cost, speed, quality, or specific capabilities",
        "Adapt to changing conditions and user preferences"
    ],
    "🔧 Extensibility": [
        "Add new AI providers without modifying existing code",
        "Plugin-like architecture for easy integration",
        "Support for custom models and local deployments"
    ],
    "🛡️ Robustness": [
        "Automatic failover and retry mechanisms",
        "Graceful degradation when providers are unavailable",
        "Environment-specific configurations"
    ],
    "💼 Production Ready": [
        "Cost tracking and optimization",
        "Performance monitoring and comparison",
        "User preference management"
    ],
    "🧪 Testing": [
        "Easy mocking for unit tests",
        "Consistent interfaces across providers",
        "Isolated testing of individual components"
    ],
    "⚡ Developer Experience": [
        "Clean, readable code with clear abstractions",
        "Configuration-driven behavior",
        "Reduced boilerplate and duplication"
    ]
}

for category, items in benefits.items():
    print(f"\n{category}")
    for item in items:
        print(f"   • {item}")

print("\n" + "="*60)
print("🚀 FACTORY PATTERN: The Foundation for Scalable AI Systems")
print("="*60)

print("\n🌟 When to Use Factory Pattern in LLM Applications:")
use_cases = [
    "Multiple AI providers or models to choose from",
    "Different requirements for different use cases", 
    "Need for runtime provider selection",
    "Environment-specific configurations",
    "Cost optimization requirements",
    "Failover and reliability needs",
    "Future extensibility requirements"
]

for i, use_case in enumerate(use_cases, 1):
    print(f"   {i}. {use_case}")

print("\n💡 Pro Tip: The Factory Pattern is particularly powerful in LLM applications")
print("   because the AI landscape is rapidly evolving. What works best today")
print("   might be different tomorrow - the Factory Pattern keeps you adaptable!")

print("\n🎉 Congratulations! You've mastered the Factory Pattern for LLM applications!")

## 🔗 Next Steps and Integration

### Integration with Other Patterns

The Factory Pattern works excellently with other design patterns:

- **Abstract Factory**: For creating families of related AI components (LLM + Embeddings + Tools)
- **Strategy Pattern**: For different algorithms/approaches within each provider
- **Builder Pattern**: For constructing complex prompts and configurations
- **Chain of Responsibility**: For routing requests through different AI agents

### Real-World Implementation Tips

1. **Configuration Management**: Use environment variables or configuration files for API keys and settings
2. **Error Handling**: Implement proper exception handling and logging
3. **Rate Limiting**: Add rate limiting to prevent API quota exhaustion
4. **Monitoring**: Track usage, costs, and performance metrics
5. **Caching**: Implement response caching for repeated queries
6. **Security**: Never hardcode API keys; use secure configuration management

### Try It Yourself!

Experiment with:
- Adding your own custom AI provider
- Implementing different selection strategies
- Building more sophisticated failover logic
- Creating domain-specific factories (e.g., for medical, legal, or financial applications)

The Factory Pattern is your gateway to building flexible, maintainable, and scalable AI applications! 🚀